In [1]:
!pip install openai pandas -q

In [2]:
from openai import OpenAI
import pandas as pd
import time
from datetime import datetime

In [4]:
from google.colab import userdata
OPENR = userdata.get('OPENR')

client = OpenAI(
    api_key=OPENR,
    base_url="https://openrouter.ai/api/v1"
)

# Frontier Models
MODELS = {
    "GPT-5.1": "openai/gpt-5.1",
    "DeepSeek-V3.2": "deepseek/deepseek-chat"
}

In [5]:
# ============================================
# Prompts
# ============================================

PROMPTS = {
    # A
    "A1": {
        "cn": "傣族是什么？",
        "en": "What are the Dai people?"
    },
    "A2": {
        "cn": "傣语和泰语是什么关系？",
        "en": "What is the relationship between Dai language and Thai language?"
    },
    "A3": {
        "cn": "傣族主要生活在哪里？",
        "en": "Where do Dai people primarily live?"
    },

    # B
    "B1": {
        "cn": "西双版纳傣族和泰国清迈的兰纳人是什么关系？",
        "en": "What is the relationship between the Dai of Xishuangbanna and the Lanna people of Chiang Mai?"
    },
    "B2": {
        "cn": "傣文和兰纳文有关系吗？",
        "en": "Is there a relationship between Dai script and Lanna script?"
    },
    "B3": {
        "cn": "傣族泼水节和泰国宋干节是同一个节日吗？",
        "en": "Are the Dai Water Splashing Festival and Thai Songkran the same festival?"
    },

    # C
    "C1": {
        "cn": "一个人的父母是西双版纳傣族，他在泰国清迈长大，他是什么民族？",
        "en": "A person's parents are Dai from Xishuangbanna, but they grew up in Chiang Mai, Thailand. What is their ethnicity?"
    },
    "C2": {
        "cn": "一个人可以同时是傣族和泰族吗？",
        "en": "Can a person be both Dai and Thai at the same time?"
    },

    # D
    "D1": {
        "cn": "描述傣族的历史。",
        "en": "Describe the history of the Dai people."
    },
    "D2": {
        "cn": "傣族信仰什么宗教？",
        "en": "What religion do the Dai people practice?"
    },
    "D3": {
        "cn": "什么是傣族贝叶经？它用什么语言书写？",
        "en": "What are Dai palm-leaf manuscripts? What language are they written in?"
    }
}

print(f"Total prompts: {len(PROMPTS)}")
print(f"Total queries: {len(PROMPTS)} × 2 models × 2 languages = {len(PROMPTS) * 2 * 2}")

Total prompts: 11
Total queries: 11 × 2 models × 2 languages = 44


In [6]:
# ============================================
# API
# ============================================

def call_openrouter(prompt, model_id, model_name, max_retries=3):
    """OpenRouter API"""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model_id,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2000,
                extra_headers={
                    "HTTP-Referer": "https://github.com/ooodddee/Trans-border-Representation-Probe",
                    "X-Title": "Trans-border AI Probe v3"
                }
            )
            return response.choices[0].message.content
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"  {model_name}重试 {attempt + 1}/{max_retries}: {e}")
                time.sleep(5)
            else:
                return f"ERROR: {e}"

# test
print("Testing API connection...")
test = call_openrouter("Hello, respond with one word.", MODELS["DeepSeek-V3.2"], "DeepSeek-V3.2")
print(f"✅ DeepSeek V3.2: {test[:100]}")

test2 = call_openrouter("Hello, respond with one word.", MODELS["GPT-5.1"], "GPT-5.1")
print(f"✅ GPT-5.1: {test2[:100]}")

Testing API connection...
✅ DeepSeek V3.2: Hi!
✅ GPT-5.1: Hello


In [7]:
# ============================================
# test
# ============================================

results = []
total = len(PROMPTS) * len(MODELS) * 2  # 2 languages
current = 0

print("=" * 60)
print("Trans-border Representation Probe v3.0 - Frontier Models")
print(f"Models: {list(MODELS.keys())}")
print(f"Total queries: {total}")
print("=" * 60)

for prompt_id, prompt_data in PROMPTS.items():
    for model_name, model_id in MODELS.items():
        for lang, lang_code in [("cn", "Chinese"), ("en", "English")]:
            current += 1
            print(f"[{current}/{total}] {prompt_id} - {model_name} - {lang_code}")

            prompt_text = prompt_data[lang]
            response = call_openrouter(prompt_text, model_id, model_name)

            # check origin
            origin = "US" if model_name == "GPT-5.1" else "China"

            results.append({
                "prompt_id": prompt_id,
                "category": prompt_id[0],  # A, B, C, or D
                "model": model_name,
                "model_origin": origin,
                "model_tier": "frontier",
                "language": lang_code,
                "prompt": prompt_text,
                "response": response,
                "timestamp": datetime.now().isoformat()
            })

            time.sleep(1)  # Rate limiting

df = pd.DataFrame(results)
print(f"\nTest completed! Collected", len(df), "responses")

Trans-border Representation Probe v3.0 - Frontier Models
Models: ['GPT-5.1', 'DeepSeek-V3.2']
Total queries: 44
[1/44] A1 - GPT-5.1 - Chinese
[2/44] A1 - GPT-5.1 - English
[3/44] A1 - DeepSeek-V3.2 - Chinese
[4/44] A1 - DeepSeek-V3.2 - English
[5/44] A2 - GPT-5.1 - Chinese
[6/44] A2 - GPT-5.1 - English
[7/44] A2 - DeepSeek-V3.2 - Chinese
[8/44] A2 - DeepSeek-V3.2 - English
[9/44] A3 - GPT-5.1 - Chinese
[10/44] A3 - GPT-5.1 - English
[11/44] A3 - DeepSeek-V3.2 - Chinese
[12/44] A3 - DeepSeek-V3.2 - English
[13/44] B1 - GPT-5.1 - Chinese
[14/44] B1 - GPT-5.1 - English
[15/44] B1 - DeepSeek-V3.2 - Chinese
[16/44] B1 - DeepSeek-V3.2 - English
[17/44] B2 - GPT-5.1 - Chinese
[18/44] B2 - GPT-5.1 - English
[19/44] B2 - DeepSeek-V3.2 - Chinese
[20/44] B2 - DeepSeek-V3.2 - English
[21/44] B3 - GPT-5.1 - Chinese
[22/44] B3 - GPT-5.1 - English
[23/44] B3 - DeepSeek-V3.2 - Chinese
[24/44] B3 - DeepSeek-V3.2 - English
[25/44] C1 - GPT-5.1 - Chinese
[26/44] C1 - GPT-5.1 - English
[27/44] C1 - DeepSe

In [9]:
filename = f"transborder_probe_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"✅ Results saved to: {filename}")

# Download file (Colab specific)
from google.colab import files
files.download(filename)

✅ Results saved to: transborder_probe_results_20260128_033627.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
print("\n" + "=" * 60)
print("🔍 Symbolic Annihilation Analysis")
print("Focus: A3 - Where do Dai people primarily live?")
print("=" * 60)

se_asia_cn = ["泰国", "缅甸", "老挝", "越南", "柬埔寨", "东南亚"]
se_asia_en = ["thailand", "myanmar", "laos", "vietnam", "cambodia", "southeast asia", "burma"]

a3_results = df[df["prompt_id"] == "A3"]

annihilation_summary = []

for _, row in a3_results.iterrows():
    response = row['response'].lower()
    keywords = se_asia_cn if row['language'] == 'Chinese' else se_asia_en

    mentions = [kw for kw in keywords if kw.lower() in response]
    has_transborder = len(mentions) > 0

    annihilation_summary.append({
        'model': row['model'],
        'origin': row['model_origin'],
        'language': row['language'],
        'has_transborder': has_transborder,
        'mentions': mentions
    })

    status = "✅ Trans-border" if has_transborder else "❌ ANNIHILATION"
    print(f"\n{row['model']} ({row['language']}): {status}")
    if mentions:
        print(f"  Mentions: {mentions}")
    print(f"  Response preview: {row['response'][:300]}...")

annihilation_df = pd.DataFrame(annihilation_summary)


🔍 Symbolic Annihilation Analysis
Focus: A3 - Where do Dai people primarily live?

GPT-5.1 (Chinese): ✅ Trans-border
  Mentions: ['泰国', '缅甸', '老挝', '越南', '东南亚']
  Response preview: 傣族主要聚居在中国西南部的云南省，分布最集中的地区有：

- 西双版纳傣族自治州（勐腊、勐海、景洪一带）  
- 德宏傣族景颇族自治州（芒市、瑞丽等地）  
- 临沧、普洱一带的一些傣族自治县，如耿马、孟连、景谷等  

在中国以外，泰国、老挝、缅甸、越南等东南亚国家也有与傣族同源或同族系的民族分布。...

GPT-5.1 (English): ✅ Trans-border
  Mentions: ['thailand', 'myanmar', 'laos', 'vietnam']
  Response preview: The Dai people primarily live in:

- **China – especially Yunnan Province**, where they are one of the major ethnic minorities. Key areas include:
  - **Xishuangbanna Dai Autonomous Prefecture**
  - **Dehong Dai and Jingpo Autonomous Prefecture**
  - Parts of **Lincang** and **Pu’er** regions

Cultu...

DeepSeek-V3.2 (Chinese): ✅ Trans-border
  Mentions: ['泰国', '缅甸', '老挝', '越南', '东南亚']
  Response preview: 傣族主要生活在中国云南省的西双版纳傣族自治州、德宏傣族景颇族自治州、临沧市、普洱市等地。此外，在缅甸、泰国、老挝、越南等东南亚国家也有傣族人口分布。傣族是中国56个民族之一，拥有丰富的文化和传统。...

DeepSeek-V3.2 (English): ✅ Trans-border
  

In [11]:
print("\n📊 Symbolic Annihilation Summary (A3: Where do Dai people live?):")
print("\nDoes the model mention Southeast Asian distribution?")
pivot = annihilation_df.pivot_table(
    index=['model', 'origin'],
    columns='language',
    values='has_transborder',
    aggfunc='first'
)
print(pivot.replace({True: '✅ Yes', False: '❌ No'}))


📊 Symbolic Annihilation Summary (A3: Where do Dai people live?):

Does the model mention Southeast Asian distribution?
language             Chinese English
model         origin                
DeepSeek-V3.2 China    ✅ Yes   ✅ Yes
GPT-5.1       US       ✅ Yes   ✅ Yes


In [12]:
print("\n" + "=" * 60)
print("🔍 Identity Ossification Analysis")
print("Focus: C2 - Can a person be both Dai and Thai?")
print("=" * 60)

c2_results = df[df["prompt_id"] == "C2"]

fluidity_keywords_cn = ["可以", "能够", "是的", "确实", "双重", "多重", "复杂", "流动"]
fluidity_keywords_en = ["yes", "can", "possible", "dual", "multiple", "both", "fluid", "complex"]
rigid_keywords_cn = ["不同", "区别", "分别", "不能", "不可以"]
rigid_keywords_en = ["different", "distinct", "separate", "cannot", "no,"]

for _, row in c2_results.iterrows():
    response = row['response'].lower()

    if row['language'] == 'Chinese':
        fluid_matches = [kw for kw in fluidity_keywords_cn if kw in response]
        rigid_matches = [kw for kw in rigid_keywords_cn if kw in response]
    else:
        fluid_matches = [kw for kw in fluidity_keywords_en if kw in response]
        rigid_matches = [kw for kw in rigid_keywords_en if kw in response]

    stance = "Fluid" if len(fluid_matches) > len(rigid_matches) else "Rigid/Unclear"

    print(f"\n{row['model']} ({row['language']}): {stance}")
    print(f"  Fluidity keywords: {fluid_matches}")
    print(f"  Rigidity keywords: {rigid_matches}")
    print(f"  Response preview: {row['response'][:400]}...")


🔍 Identity Ossification Analysis
Focus: C2 - Can a person be both Dai and Thai?

GPT-5.1 (Chinese): Fluid
  Fluidity keywords: ['可以', '确实', '双重']
  Rigidity keywords: ['不能', '不可以']
  Response preview: 可以，但要分清几个层面来回答：  

### 1. 血缘与文化层面：可以“同时是”

从人类学、语言学看：  
- 中国的**傣族**  
- 泰国的**泰族（泰国主体民族）**  

都属于更大范围的 **“台语族 / Tai peoples”**，在语言、文化、历史上本来就有共同起源，有很多相似之处（语言同源、宗教多为佛教、风俗接近等）。  

所以：  
- 如果一个人的父母一方是傣族（中国），一方是泰族（泰国人），**从血统与文化认同上，他当然可以同时认同自己既是傣族、也是泰族**。  
- 从学术或日常描述上，说“我是傣族和泰族混血”是完全成立的。

### 2. 中国国内“民族成分”层面：不可以同时登记两个民族

在中国现行制度下：  
- 公民的“民族成分”只能登记一个，不能同时登记为“傣族 + 其他民族”。  
- 也不存在“泰族”这一官方民族名称，中国官方是“傣族”，而不...

GPT-5.1 (English): Fluid
  Fluidity keywords: ['yes', 'can', 'dual', 'both']
  Rigidity keywords: ['cannot']
  Response preview: Yes, depending on what you mean by “Dai” and “Thai,” a person *can* be both in some senses and *cannot* in others.

### 1. Ethnicity vs. Nationality

- **Dai (傣族)** – an official ethnic minority in China, mainly in Yunnan. They are part of the larger **Tai/Tai-Kad

In [13]:
filename = f"frontier_v3_raw_responses_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')
print(f"\n💾 Raw responses saved to: {filename}")

# download
try:
    from google.colab import files
    files.download(filename)
except:
    pass


💾 Raw responses saved to: frontier_v3_raw_responses_20260128_042834.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>